In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Unsupervised Learning

As a first step, we will need to import the data from the "`online_retail.csv`" data file that was preprocessed for us so that features are properly scaled.

We also want to define the column that we are going to use as the row labels of the DataFrame; in this case, *'CustomerID'*. Once loaded, we can apply once more the `head()` function to preview the first 5 rows of our DataFrame. 

In [ ]:
# Import the data from the online_retail.csv, 
# set the index column and explore the first few rows
customers = pd.read_csv('data/online_retail.csv', index_col='CustomerID')
customers.head()


## Clustering with K-Means

K-means clustering is a method for finding clusters and cluster centers in a set of points. The K-means algorithm alternates the two steps:

1. for each center, identify the subset of training points that are closer to it than to any other center
2. update the location of the center to match the points related to it

These two steps are iterated until the centers no longer move (significantly) or the assignments no longer change. Then, a new point $x$ can be assigned to the cluster of the closest prototype.

### Learning Activity - Run K-Means with two features

Isolate the features `mean_spent` and `max_spent`, then run the K-Means algorithm on the resulting dataset using K=2 and visualise the result. You will need:

* to create an instance of `KMeans` with 2 clusters
* fit this to the isolated features (via the `.fit` method)
* look how it's doing by using by showing the assignment predicted (via the `.predict` method)

This is the standard SKLearn workflow for most of the algorithms.

In [ ]:
from sklearn.cluster import KMeans

# Apply k-means with 2 clusters using a subset of features 
# (mean_spent and max_spent)
kmeans = KMeans(n_clusters = 2)
cust2  = customers[['mean_spent', 'max_spent']]
kmeans.fit(cust2)
cluster_assignment = kmeans.predict(cust2)


Let us introduce a simple function to better visualise what's going on:

In [ ]:
# This function generates a pairplot enhanced with the result of k-means
def pairplot_cluster(df, cols, cluster_assignment):
    """
    Input
        df, dataframe that contains the data to plot
        cols, columns to consider for the plot
        cluster_assignments, cluster asignment returned 
        by the clustering algorithm
    """
    # seaborn will color the samples according to the column cluster
    df['cluster'] = cluster_assignment 
    sns.pairplot(df, vars=cols, hue='cluster', size=6)
    df.drop('cluster', axis=1, inplace=True)

And let's use it now to see how we did previously... (ignore the warnings if anything comes up)

In [ ]:
# Visualise the clusters using pairplot_cluster()
pairplot_cluster(customers, ['mean_spent', 'max_spent'], cluster_assignment)


#### What can you observe?

* the separation between the two clusters is "clean" (the two clusters can be separated with a line)
* one cluster contains customers with low spendings, the other one with high spendings

### Test Activity - Run K-Means with all the features
Run K-Means using all the features available and visualise the result in the subspace `mean_spent` and `max_spent`.

In [ ]:
# Apply k-means with 2 clusters using all features
kmeans = KMeans(n_clusters = 2)
kmeans.fit(customers)
cluster_assignment = kmeans.predict(customers)


and visualise using the same subset of variables as before... what has changed??

In [ ]:
# Visualise the clusters using pairplot_cluster()
pairplot_cluster(customers, ['mean_spent', 'max_spent'], cluster_assignment)


***Question***: Why can't the clusters be separated with a line as before?

### Learning activity - Compare expenditure between clusters

Select the features `'mean_spent'` and `'max_spent'` and compare the two clusters obtained above using them. 

To do so, filter `customers` to keep only the rows associated to a given cluster, then apply `.describe()` on the result. That will return a DataFrame object containing summary statistics, you can save it in a new variable. If you do so for both clusters, you can then concatenate the two variables in one single DataFrame and easily compare the statistics per cluster.

In [ ]:
# Compare expenditure between clusters
features = ['mean_spent', 'max_spent']

# create a dataframe corresponding to the case
# cluster_assignment == 0
cluster_0 = customers.loc[cluster_assignment == 0, features].describe()
cluster_0.columns = [c + '_0' for c in cluster_0.columns]


In [ ]:
# then with ==1
cluster_1 = customers.loc[cluster_assignment == 1, features].describe()
cluster_1.columns = [c + '_1' for c in cluster_1.columns]


In [ ]:
#Concatenate both:
compare_df = pd.concat([cluster_0, cluster_1], axis=1)
compare_df


### Test Activity - Looking at the centroids

Look at the centroids of the clusters `kmeans.cluster_centers_` and check the values of the centers in for the features `mean_spent`, `max_spent`. You will need to create a new DataFrame where the data is simply `kmeans.cluster_centers_`.

In [ ]:
# Get the centroids and display them
centers_df = pd.DataFrame(data=kmeans.cluster_centers_, columns=customers.columns)
print(centers_df[features])


### Learning Activity - Compare mean expediture with box plot

Compare the distribution of the feature `mean_spent` in the two clusters using a box plot. You will need:

* `sns.boxplot` (seaborn's boxplot)

In [ ]:
# Compare mean expediture with box plot
sns.boxplot(data=pd.DataFrame({"mean_spent_0": customers.loc[cluster_assignment == 0, "mean_spent"], 
                               "mean_spent_1": customers.loc[cluster_assignment == 1, "mean_spent"]}), 
            showfliers=False)


does this seem to make sense? How can you interpret the plots?

### Learning Activity - Compute the silhouette score
Compute the silhouette score of the clusters resuting from the application of K-Means.

The Silhouette Coefficient is calculated using the mean intra-cluster distance (``a``) and the mean nearest-cluster distance (``b``) for each sample.  The Silhouette Coefficient for a sample is ``(b - a) / max(a, b)``. It represents how similar a sample is to the samples in its own cluster compared to samples in other clusters.

The best value is 1 and the worst value is -1. Values near 0 indicate overlapping clusters. Negative values generally indicate that a sample has been assigned to the wrong cluster, as a different cluster is more similar.

SKLearn provides the function `silhouette_score` which you can call and display.

In [ ]:
from sklearn.metrics import silhouette_score

# Computing the silhouette score
print('silhouette_score {0:.2f}'.format(silhouette_score(customers, cluster_assignment)))


This silhouette score is reasonably high which we can intepret by saying that the corresponding clusters are quite compact.

## Hierarchical clustering: Linking with Linkage

The main idea behind hierarchical clustering is that you start with each point in it's own cluster and then

1. compute distances between all clusters
2. merge the closet clusters

Do this repeatedly until you have only one cluster.

This algorithm groups the samples in a bottom-up fashion and falls under the category of the agglomerative clustering algorithms.

According to the distance between clusters and samples that one choose the clusters will have different properties. In this section we'll use a distance that will minimizes the variance of the clusters being merged.

This algorithm results in a hierarchy, or binary tree, of clusters branching down to the last layer which has a leaf for each point in the dataset that can be visualise with a "Dendrogram". The advantage of this approach is that clusters can grow according to the shape of the data rather than being globular.

sklearn implements hierarchical clustering in the class `sklearn.cluster.AgglomerativeClustering` (http://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering), this class is mainly a wrapper to the functions in `scipy.cluster.hierarchy` (http://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html).

### Learning Activity - Plotting dendograms
Use the function `linkage()` from `scipy.cluster.hierarchy` to cluster the retail data and pass the result to the function `dendrogram()` to visualise the result. Truncate the dendrogram if the initial result is unreadable.

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram

# Apply hierarchical clustering 
Z = linkage(customers, method='ward', metric='euclidean')

# Draw the dendrogram
plt.figure(figsize=(18,6))
dendrogram(Z)
plt.ylabel('Distance')
plt.xlabel('Sample index')


The coloring of the figure highlights that the data can be segmented in two big clusters that were merged only in the very last iterations of the algorithm.

We can improve the readability of the dendrogram showing only the last merged clusters and a threshold to color the clusters. For this use

* the option `truncate_mode` in `dendrogram`.

In [ ]:
# Draw the dendrogram using a cut_off value
plt.figure(figsize=(18,6))
cut_off = 60

dendrogram(Z, color_threshold=cut_off, truncate_mode='lastp', p=20)

plt.ylabel('Distance')
plt.xlabel('Sample index or (cluster size)')
plt.hlines(cut_off, 0, len(customers), linestyle='--')


oh yeah that's a lot better!

### Learning Activity - Running Linkage with Sklearn

Use `sklearn.cluster.AgglomerativeClustering` to cluster the retail data according to the 3 clusters highlighted by the dendrogram above and visualise the result in the subspace give by the features `mean_spent` and `max_spent`.

In [ ]:
from sklearn.cluster import AgglomerativeClustering

# Perform clustering with AgglomerativeClustering
# Ward merges two clusters if the resulting has small variance

n_cluster = 3
agglomerative = AgglomerativeClustering(
                    n_clusters=n_cluster, linkage='ward', 
                    affinity='euclidean')

cluster_assignment = agglomerative.fit_predict(customers)


Let's visualise this with the `pairplot_cluster()`

In [ ]:
# Visualise the clusters using pairplot_cluster()
pairplot_cluster(customers, ['mean_spent', 'max_spent'], cluster_assignment)


## DBSCAN

The DBSCAN algorithm views clusters as areas of high density separated by areas of low density. Due to this rather generic view, clusters found by DBSCAN can be any shape, as opposed to k-means which assumes that clusters are globular. The central component to the DBSCAN is the concept of core samples, which are samples that are in areas of high density. A cluster is therefore a set of core samples, each close to each other (measured by some distance measure) and a set of non-core samples that are close to a core sample (but are not themselves core samples). There are two parameters to the algorithm, min_samples and eps, which define formally what we mean when we say dense. Higher min_samples or lower eps indicate higher density necessary to form a cluster. 

Summary of the Algorithm:

- starts with an arbitrary starting point and retrieved all the points in the radius of distance `eps` from it 
    - if the radius contains `min_samples` points, start a cluster
      - add all the points in the radius of distance `eps` to the cluster and their `eps` neighbors.
      - continue expanding the cluster iterating on the the procedure on all the neighbors
    - otherwise mark it as noise/outlier

Sklearn implementation doc: http://scikit-learn.org/stable/modules/clustering.html#dbscan

Animated DBSCAN: http://www.naftaliharris.com/blog/visualizing-dbscan-clustering/

## Learning Activity - A starting value for eps

Measure the distance of each point to its closest neighbor using the function `sklearn.metrics.pairwise.pairwise_distances` (http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html) and plot the distribution of the distances.

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

# Compute all the pairwise distances
all_distances = pairwise_distances(customers, metric='euclidean')


In [ ]:
# compute the distance of each point to its closest neighbor
neig_distances = [np.min(row[np.nonzero(row)]) for row in all_distances]


In [ ]:
# Plot the distances
plt.hist(neig_distances, bins=50)
plt.xlabel('Distance from closest sample')
plt.ylabel('Occurrences')
plt.axis([0,1.5,0,2500])


The distribution of the distance will help us choose a starting point for `eps`. We see that it's very likely that a point as at least one neighbour in a radius of 0.15 and that only very few point have one at distance over 1.0. Since we want that a core point has more than one point in is `eps`-neighborhood we can start picking `eps` on the right tail of the distribution.

## Learning Activity - Applying DBSCAN

Cluster the customer data with DBSCAN and visualise the results in the subspaces used for the other algorithms.

In [ ]:
from sklearn.cluster import DBSCAN

# Apply DBSCAN setting eps to 1.0 and min samples to 8 (say)
db = DBSCAN(eps=1.0, min_samples=8)
cluster_assignment = db.fit_predict(customers)


In [ ]:
# Display how many clusters were found
clusters_found = np.unique(cluster_assignment)
print ('Clusters found', len(clusters_found))


We can then visualise this

In [ ]:
# Visualise the clusters using pairplot_cluster()
pairplot_cluster(customers, ['mean_spent', 'max_spent'], cluster_assignment)


DBSCAN clustered all the points in one big cluster and marked as outiers all the points that are not in dense areas.

### Learning Activity -  How many clusters with DBSCAN?

Vary `eps` and `min_samples` and study how the number of clusters varies as result. This way we'll have an idea of how many cluster we get varying the parameters. This can help us choose the parameters if we already have an idea of how many clusters we want to create.

Warning, if you cover a grid of points, it may take a while to finish, don't put too many points!

In [ ]:
# WARNING this may take a couple of minutes to finish!
eps = np.linspace(.5, 10.0, 5)
mins = np.arange(5, 50, 5)
Z = np.zeros((len(eps), len(mins)))

for i, e in enumerate(eps):
    for j, m in enumerate(mins):
        db   = DBSCAN(eps=e, min_samples=m)
        pred = db.fit_predict(customers)
        clusters_found = len(np.unique(pred))
        Z[i,j] = clusters_found

In [ ]:
# Visualise this using a heatmap
plt.figure(figsize=(10, 10))
sns.heatmap(Z, cmap='RdBu', center=0, annot=True);
plt.xticks(np.arange(Z.shape[1]), mins)
plt.xlabel('min_samples')
plt.yticks(np.arange(Z.shape[0]), ['%0.2f' % x for x in eps])
plt.ylabel('eps')


### Learning activity - Compute the silhouette score of the DBSCAN cluster

Compute the silhouette score of the clusters made with DBSCAN and compare it with the silhouette score achieved with K-Means.

In [ ]:
# Compute the silhouette score of DBSCAN
print(silhouette_score(customers, cluster_assignment))
